In [1]:
import requests
import json
import time
from datetime import datetime
import pymysql

import sys
import pandas as pd

In [2]:
def check_eqkmsg(mykey, endpoint, stday):
    params = {'ServiceKey':mykey, 'fromTmFc':stday, 'toTmFc':stday,
            'dataType':'json', 'pageNo':1, 'numOfRows':10}
    try:
        r = requests.get(endpoint+'/'+'getEqkMsg', params=params)
    except:
        print('기상청 API 호출에 실패했습니다!')
        sys.exit(1)

    if r.status_code != 200:
        if r.status_code == 401:
            r = requests.get(endpoint+'/'+'getEqkMsg', params=params)
        elif r.status_code == 429:
            time.sleep(1)
            r = requests.get(endpoint+'/'+'getEqkMsg', params=params)
        else:
            print('기상청 API 호출에러 발생!:',r.status_code)

    res = json.loads(r.text).get('response')
    rcode = res.get('header')['resultCode']
    if rcode == '00':
        rescount = res.get('body')['totalCount']
        rpage = (rescount//10)+1
    elif rcode == '03':
        print('오늘의 지진 통보문이 없습니다. ({})'.format(rcode))
        sys.exit(1)
    else:
        return res, rpage

In [3]:
def get_eqkmsg(mykey, endpoint, stday, npage):
    params = {'ServiceKey':mykey, 'fromTmFc':stday, 'toTmFc':stday,
            'dataType':'json', 'pageNo':str(npage), 'numOfRows':10}
    try:
        r = requests.get(endpoint+'/'+'getEqkMsg', params=params)
    except:
        print('기상청 API 호출에 실패했습니다!')
        sys.exit(1)

    if r.status_code != 200:
        if r.status_code == 401:
            r = requests.get(endpoint+'/'+'getEqkMsg', params=params)
        elif r.status_code == 429:
            time.sleep(1)
            r = requests.get(endpoint+'/'+'getEqkMsg', params=params)
        else:
            print('기상청 API 호출에러 발생!:',r.status_code)

    try:
        res = json.loads(r.text).get('response')
    except:
        print('fail')
        res=None
    return res


In [4]:
def get_eqklist(stday, data):
    raw = data
    eqk_data = raw.get('body').get('items')['item']
    eqk_list = []
    for i in range(0, len(eqk_data)):
        eqk_dict = {}
        pick = eqk_data[i]
        eqk_dict['id'] = stday+'-'+str(raw.get('body')['pageNo'])+str(i)
        eqk_dict['day'] = str(pick.get('tmEqk'))[0:8]
        eqk_dict['eqkt'] = str(pick.get('tmEqk'))[8:14]
        eqk_dict['fct'] = str(pick.get('tmFc'))[8:12]
        eqk_dict['mt'] = pick.get('mt')
        eqk_dict['loc'] = pick.get('loc')
        eqk_dict['lat'] = pick.get('lat')
        eqk_dict['lon'] = pick.get('lon')
        eqk_dict['mapURL'] = pick.get('img')
        eqk_dict['fctype'] = pick.get('fcTp')
        eqk_dict['depth'] = pick.get('dep')
        eqk_list.append(eqk_dict)
    time.sleep(1)
    return eqk_list

In [41]:
def insert_db(cursor, sql, data):
    for i in range(0, len(data)):
        eqkdata = data[i]
        cursor.execute(sql, list(eqkdata.values())*2)

In [5]:
def main():
    host = 'db-1.crtxwdze1fvo.ap-northeast-2.rds.amazonaws.com'
    port = 3306
    user = 'blackbird1227'
    db = 'api1'
    pw = 'gksqls1227!'
    conn = pymysql.connect(host=host, user=user, passwd=pw, db=db, port=port, use_unicode=True, charset='utf8')
    table = 'dayeqkinfo'
    cursor = conn.cursor()

    mykey = 'hLgHjx4SRbAp3W6wWAgqvPqZB0dDxQlzAPhuLVz1nCYJ9WUW//16Hzz6vy/0n7rE4kB1kJ3/Ia7vxld3oid6IA=='
    endpoint = 'http://apis.data.go.kr/1360000/EqkInfoService'
    today = str(datetime.today())[0:10].split('-')
    stday =  today[0]+today[1]+today[2]
    npage = 1
    res, rpage = check_eqkmsg(mykey, endpoint, stday)
    
    if rpage != 1:
        eqk_list = get_eqklist(stday, res)
        while npage < rpage:
            time.sleep(1)
            npage = npage +1
            res = get_eqkmsg(mykey, endpoint, stday, npage)
            bind = get_eqklist(stday, res)
            eqk_list = eqk_list.append(bind)
    else:
        eqk_list = get_eqklist(stday, res)

    col = ', '.join(eqk_list[0].keys())
    p_holders = ', '.join(['%s']*len(eqk_list[0]))
    k_holders = ', '.join([i+'=%s' for i in eqk_list[0].keys()])
    sql = 'INSERT INTO {} ({}) VALUES ({}) ON DUPLICATE KEY UPDATE {}'.format(table, col, p_holders, k_holders)
    insert_db(cursor, sql, eqk_list)
    cursor.fetchall()
    conn.commit()
    cursor.close()
    

In [6]:
if __name__ == '__main__':
    main()

오늘의 지진 통보문이 없습니다. (03)


SystemExit: 1

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
mykey = 'hLgHjx4SRbAp3W6wWAgqvPqZB0dDxQlzAPhuLVz1nCYJ9WUW//16Hzz6vy/0n7rE4kB1kJ3/Ia7vxld3oid6IA=='
endpoint = 'http://apis.data.go.kr/1360000/EqkInfoService'
today = str(datetime.today())[0:10].split('-')
stday =  today[0]+today[1]+today[2]
npage = 1
params = {'ServiceKey':mykey, 'fromTmFc':'20220427', 'toTmFc':stday,
            'dataType':'json', 'pageNo':npage, 'numOfRows':10}
r = requests.get(endpoint+'/'+'getEqkMsg', params=params)
raw = json.loads(r.text).get('response')
raw

{'header': {'resultCode': '00', 'resultMsg': 'NORMAL_SERVICE'},
 'body': {'dataType': 'JSON',
  'items': {'item': [{'cnt': 1,
     'fcTp': 2,
     'img': 'http://www.weather.go.kr/repositary/image/eqk/img/eqk_img_2_20220428222113.png',
     'inT': '',
     'lat': -3.95,
     'loc': '파푸아뉴기니 마당 북동쪽 170km 해역',
     'lon': 146.66,
     'mt': 6.0,
     'rem': '국내영향없음. 위 자료는 미지질조사소(USGS) 분석결과임.',
     'stnId': 108,
     'tmEqk': 20220428222113,
     'tmFc': 202204282249,
     'tmSeq': 415,
     'dep': 10}]},
  'pageNo': 1,
  'numOfRows': 10,
  'totalCount': 1}}

In [6]:
eqkdata = raw.get('body').get('items')['item']
eqk_dict = {}
eqk_list = []
for i in range(0, len(eqkdata)):
    pick = eqkdata[i]
    eqk_dict['id'] = str(pick.get('tmEqk'))[0:8]+'-'+str((raw.get('body')['pageNo'])-1)+str(i)
    eqk_dict['day'] = str(pick.get('tmEqk'))[0:8]
    eqk_dict['eqkt'] = str(pick.get('tmEqk'))[8:14]
    eqk_dict['fct'] = str(pick.get('tmFc'))[8:12]
    eqk_dict['mt'] = pick.get('mt')
    eqk_dict['loc'] = pick.get('loc')
    eqk_dict['lat'] = pick.get('lat')
    eqk_dict['lon'] = pick.get('lon')
    eqk_dict['mapURL'] = pick.get('img')
    eqk_dict['fctype'] = pick.get('fcTp')
    eqk_dict['depth'] = pick.get('dep')
    eqk_list.append(eqk_dict)
time.sleep(1)
eqk_list

[{'id': '20220428-00',
  'day': '20220428',
  'eqkt': '222113',
  'fct': '2249',
  'mt': 6.0,
  'loc': '파푸아뉴기니 마당 북동쪽 170km 해역',
  'lat': -3.95,
  'lon': 146.66,
  'mapURL': 'http://www.weather.go.kr/repositary/image/eqk/img/eqk_img_2_20220428222113.png',
  'fctype': 2,
  'depth': 10}]

In [7]:
host = 'db-1.crtxwdze1fvo.ap-northeast-2.rds.amazonaws.com'
port = 3306
user = 'blackbird1227'
db = 'api1'
pw = 'gksqls1227!'
conn = pymysql.connect(host=host, user=user, passwd=pw, db=db, port=port, use_unicode=True, charset='utf8')
table = 'dayeqkinfo'
cursor = conn.cursor()

col = ', '.join(eqk_list[0].keys())
p_holders = ', '.join(['%s']*len(eqk_list[0]))
k_holders = ', '.join([i+'=%s' for i in eqk_list[0].keys()])
sql = 'INSERT INTO {} ({}) VALUES ({}) ON DUPLICATE KEY UPDATE {}'.format('dayeqkinfo', col, p_holders, k_holders)
cursor.execute(sql, list(eqk_list[0].values())*2)
cursor.fetchall()

()

In [8]:
conn.commit()
cursor.close()